In [1]:
# caculate the weather data of each outbreak

In [176]:
import pandas as pd
import numpy as np
import datetime

In [222]:
salm_station = pd.read_csv('/Users/evelyn/Desktop/weather information/salm_station.csv')

In [291]:
salm = pd.read_excel('/Users/evelyn/Desktop/project/Cleaned data - Salmonella and norovirus.xlsx', sheet_name = 'Salmonella')

In [223]:
salm_station = salm_station.dropna(subset = ['USAF'])
salm_station = salm_station.reset_index(drop = True)

In [224]:
salm_station["WBAN"] = pd.to_numeric(salm_station["WBAN"], errors='coerce').fillna('0').astype('int32')
for a in range(len(salm_station)):
    wban_unmodified = salm_station.loc[a, "WBAN"]
    if 0 < wban_unmodified < 10000: 
        salm_station.loc[a, "WBAN"] = str(wban_unmodified).zfill(5)


In [304]:
salm_null = pd.read_csv('/Users/evelyn/Desktop/weather information/salm_interpolated stations.csv')

In [310]:
for z in range(len(salm_null)):
    salm_null.loc[z, "USAF0"] = str(salm_null.loc[z, "USAF0"])
    salm_null.loc[z, "WBAN0"] = str(int(salm_null.loc[z, "WBAN0"]))

    if not(pd.isna(salm_null.loc[z, "USAF1"])):
        salm_null.loc[z, "USAF1"] = str(salm_null.loc[z, "USAF1"])
        salm_null.loc[z, "WBAN1"] = str(int(salm_null.loc[z, "WBAN1"]))
        
    if not(pd.isna(salm_null.loc[z, "USAF2"])):
        salm_null.loc[z, "USAF2"] = str(salm_null.loc[z, "USAF2"])
        salm_null.loc[z, "WBAN2"] = str(int(salm_null.loc[z, "WBAN2"]))

In [294]:
path = '/Users/evelyn/Desktop/weather information/salm-weather/'

path1 = '/Users/evelyn/Desktop/weather information/salm_interpolated/'

import os
import os.path
import numpy as np


for i in range(len(salm)):
    print('{:.4%}'.format(i/len(salm)))
    
    df_outbreak = salm_station[salm_station["CDCID"] == salm.loc[i,"CDCID"]]
    df_outbreak = df_outbreak.reset_index(drop = True)
    
    df_null = salm_null[salm_null["CDCID"] == salm.loc[i,"CDCID"]]
    df_null = df_null.reset_index(drop = True)
    
    TEMP = [] ; MAX = [] ; MIN = [] ; DEWP = [] ; SLP = [] ; STP = [] ; WDSP = [] ; MXSPD = [];
    PRCP = [] ; SNDP = [];
    Fog = [] ; Rain = [] ; Snow = [] ; Hail = [] ; Thunder = [] ; Tornado = [];
    if (len(df_outbreak) > 0):
        startdate = datetime.datetime.strptime(df_outbreak.loc[0, "InitialExposure"], "%Y-%m-%d")
        enddate = datetime.datetime.strptime(df_outbreak.loc[0, "LastExposure"], "%Y-%m-%d")
    elif (len(df_null) > 0):
        startdate = datetime.datetime.strptime(df_null.loc[0, "InitialExposure"], "%Y-%m-%d")
        enddate = datetime.datetime.strptime(df_null.loc[0, "LastExposure"], "%Y-%m-%d")
    else:
        continue
        
    delta = datetime.timedelta(days=1)
    d = startdate
    
    while d <= enddate:
        yr = int(str(d)[:4])
        TEMP1 = [] ; MAX1 = [] ; MIN1 = [] ; DEWP1 = [] ; SLP1 = [] ; STP1 = [] ; WDSP1 = [] ; MXSPD1 = [];
        PRCP1 = [] ; SNDP1 = [];
        Fog1 = [] ; Rain1 = [] ; Snow1 = [] ; Hail1 = [] ; Thunder1 = [] ; Tornado1 = [];
        
        if (len(df_outbreak) > 0):
            for j in range(len(df_outbreak)):
                usaf = df_outbreak.loc[j, "USAF"]
                wban = df_outbreak.loc[j, "WBAN"]
                filename = str(usaf) + str(wban) + "-" + str(yr) + "DAILY.csv"
                if not(os.path.isfile(path+filename)):
                    continue 
                elif(sum(1 for line in open(path+filename)) < 10):
                    continue 
                else:
                    df = pd.read_csv(path + filename)
                    if(df.DATE.str.contains(str(d)[:10]).any()):
                        m = df[df.DATE == str(d)[:10]].index.tolist()[0]
                        if(df.loc[m, "TEMP"] < 9999):
                            TEMP1.append(df.loc[m, "TEMP"])
                        if(df.loc[m, "MAX"] < 9999):
                            MAX1.append(df.loc[m, "MAX"])
                        if(df.loc[m, "MIN"] < 9999):
                            MIN1.append(df.loc[m, "MIN"])
                        if(df.loc[m, "DEWP"] < 9999):
                            DEWP1.append(df.loc[m, "DEWP"])
                        if(df.loc[m, "SLP"] < 9999):
                            SLP1.append(df.loc[m, "SLP"])
                        if(df.loc[m, "STP"] < 9999):
                            STP1.append(df.loc[m, "STP"])
                        if(df.loc[m, "WDSP"] < 999):
                            WDSP1.append(df.loc[m, "WDSP"])
                        if(df.loc[m, "MXSPD"] < 998):
                            MXSPD1.append(df.loc[m, "MXSPD"])
                        if(df.loc[m, "PRCP"] < 99):
                            PRCP1.append(df.loc[m, "PRCP"])

                        SNDP1.append(df.loc[m, "SNDP"]) 
                        Fog1.append(df.loc[m, "Fog"])
                        Rain1.append(df.loc[m, "Rain or Drizzle"])
                        Snow1.append(df.loc[m, "Snow or Ice Pellets"])
                        Hail1.append(df.loc[m, "Hail"])
                        Thunder1.append(df.loc[m, "Thunder"])
                        Tornado1.append(df.loc[m, "Tornado or Funnel Cloud"])
                    else:
                        continue
        if (len(df_null) > 0):
            for k in range(len(df_null)):
                usaf1 = df_null.loc[k, "USAF0"]
                wban1 = df_null.loc[k, "WBAN0"]
                filename1 = str(usaf1) + str(wban1) + "-" + str(yr) + "DAILY.csv"
                dfn = pd.read_csv(path1 + filename1)
                if(dfn.DATE.str.contains(str(d)[:10]).any()):
                    a = dfn[dfn.DATE == str(d)[:10]].index.tolist()[0]
                    if(dfn.loc[a, "TEMP"] < 9999):
                        TEMP1.append(dfn.loc[a, "TEMP"]) 
                    if(dfn.loc[a, "MAX"] < 9999):
                        MAX1.append(dfn.loc[a, "MAX"])
                    if(dfn.loc[a, "MIN"] < 9999):
                        MIN1.append(dfn.loc[a, "MIN"])
                    if(dfn.loc[a, "DEWP"] < 9999):
                        DEWP1.append(dfn.loc[a, "DEWP"])
                    if(dfn.loc[a, "SLP"] < 9999):
                        SLP1.append(dfn.loc[a, "SLP"])
                    if(dfn.loc[a, "STP"] < 9999):
                        STP1.append(dfn.loc[a, "STP"])
                    if(dfn.loc[a, "WDSP"] < 999):
                        WDSP1.append(dfn.loc[a, "WDSP"])
                    if(dfn.loc[a, "MXSPD"] < 998):
                        MXSPD1.append(dfn.loc[a, "MXSPD"])
                    if(dfn.loc[a, "PRCP"] < 99):
                        PRCP1.append(dfn.loc[a, "PRCP"])
                        
                    SNDP1.append(dfn.loc[a, "SNDP"])
                    Fog1.append(dfn.loc[a, "Fog"])
                    Rain1.append(dfn.loc[a, "Rain or Drizzle"])
                    Snow1.append(dfn.loc[a, "Snow or Ice Pellets"])
                    Hail1.append(dfn.loc[a, "Hail"])
                    Thunder1.append(dfn.loc[a, "Thunder"])
                    Tornado1.append(dfn.loc[a, "Tornado or Funnel Cloud"])
                else:
                    continue
            
        if not(len(TEMP1) == 0):
            TEMP.append(np.mean(TEMP1))
        if not(len(MAX1) == 0):
            MAX.append(np.mean(MAX1))
        if not (len(MIN1) == 0):
            MIN.append(np.mean(MIN1))
        if not (len(DEWP1) == 0):
            DEWP.append(np.mean(DEWP1))
        if not (len(SLP1) == 0):
            SLP.append(np.mean(SLP1))
        if not (len(STP1) == 0):
            STP.append(np.mean(STP1))
        if not (len(WDSP1) == 0):
            WDSP.append(np.mean(WDSP1))
        if not (len(MXSPD1) == 0):
            MXSPD.append(np.mean(MXSPD1))
        if not (len(PRCP1) == 0):
            PRCP.append(np.mean(PRCP1))
        if not (len(SNDP1) == 0):
            SNDP.append(np.mean(SNDP1))
        if not (len(Fog1) == 0):
            Fog.append(np.mean(Fog1))
        if not (len(Rain1) == 0):
            Rain.append(np.mean(Rain1))
        if not (len(Snow1) == 0):
            Snow.append(np.mean(Snow1))
        if not (len(Hail1) == 0):
            Hail.append(np.mean(Hail1))
        if not (len(Thunder1) == 0):
            Thunder.append(np.mean(Thunder1))
        if not (len(Tornado1) == 0):
            Tornado.append(np.mean(Tornado1))
        
        d = d + delta
        
    salm.loc[i, "TEMP"] = np.mean(TEMP); 
    salm.loc[i, "MAX"] = np.mean(MAX); 
    salm.loc[i, "MIN"] = np.mean(MIN);
    salm.loc[i, "DEWP"] = np.mean(DEWP); 
    salm.loc[i, "SLP"] = np.mean(SLP); 
    salm.loc[i, "STP"] = np.mean(STP);
    salm.loc[i, "WDSP"] = np.mean(WDSP); 
    salm.loc[i, "MXSPD"] = np.mean(MXSPD); 
    salm.loc[i, "PRCP"] = np.mean(PRCP);
    salm.loc[i, "SNDP"] = np.mean(SNDP); 
    salm.loc[i, "Fog"] = np.mean(Fog); 
    salm.loc[i, "Rain or Drizzle"] = np.mean(Rain);
    salm.loc[i, "Snow or Ice Pellets"] = np.mean(Snow); 
    salm.loc[i, "Hail"] = np.mean(Hail); 
    salm.loc[i, "Thunder"] = np.mean(Thunder); 
    salm.loc[i, "Tornado or Funnel Cloud"] = np.mean(Tornado);  
    

0.0000%
0.1063%
0.2125%
0.3188%
0.4251%
0.5313%
0.6376%
0.7439%
0.8502%
0.9564%
1.0627%
1.1690%
1.2752%


/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1.3815%
1.4878%
1.5940%
1.7003%
1.8066%
1.9129%
2.0191%
2.1254%
2.2317%
2.3379%
2.4442%
2.5505%
2.6567%
2.7630%
2.8693%


/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


2.9756%
3.0818%
3.1881%
3.2944%
3.4006%
3.5069%
3.6132%
3.7194%


...

98.0871%
98.1934%
98.2997%
98.4060%
98.5122%


/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


98.6185%
98.7248%
98.8310%


/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


98.9373%
99.0436%
99.1498%
99.2561%
99.3624%
99.4687%
99.5749%
99.6812%
99.7875%
99.8937%


In [296]:
salm.to_csv('/Users/evelyn/desktop/test2.csv')

In [297]:
salm.isnull().sum()

CDCID                        0
GenusName                    0
InitialExposure              0
LastExposure                 0
TotalCases                   0
Exposurestate                1
ExposureState                0
ExposureCounty1              0
ExposureCounty2            841
ExposureCounty3            867
ExposureCounty4            890
ExposureCounty5            901
ExposureCounty6            909
ExposureCounty7            919
ExposureCounty8            924
ExposureCounty9            927
ExposureCounty10           928
ExposureCounty11           931
ExposureCounty12           932
ExposureCounty13           936
ExposureCounty14           938
ExposureCounty15           938
ExposureCounty16           938
ExposureCounty17           939
ExposureCounty18           939
ExposureCounty19           940
TEMP                        10
MAX                         10
MIN                         10
DEWP                        21
SLP                        138
STP                         10
WDSP    

In [320]:
salm = pd.read_csv('/Users/evelyn/Desktop/Salmonella.csv')

In [321]:
b = [16626, 276953] #have direct usaf and wban but failed to capture data during outbreaks
c = [2539, 16210, 16238, 16626, 20216, 274650,276953, 282670, 297118, 298455]
path = '/Users/evelyn/Desktop/project/salm_null/'
for i in range(len(c)):
    TEMP2 = [] ; MAX2 = [] ; MIN2 = [] ; DEWP2 = [] ; SLP2 = [] ; STP2 = [] ; WDSP2 = [] ; MXSPD2 = [];
    PRCP2 = [] ; SNDP2= [];
    Fog2 = [] ; Rain2 = [] ; Snow2 = [] ; Hail2 = [] ; Thunder2 = [] ; Tornado2 = [];
    index = salm[salm.CDCID == c[i]].index.tolist()[0]
    k = salm_null[salm_null.CDCID == c[i]].index.tolist()[0]
    
    startdate = datetime.datetime.strptime(salm_null.loc[k, "InitialExposure"], "%Y-%m-%d")
    enddate = datetime.datetime.strptime(salm_null.loc[k, "LastExposure"], "%Y-%m-%d")
    d = startdate
    delta = datetime.timedelta(days=1)
    
    
    while d <= enddate:
        yr = int(str(d)[:4])
        w = [salm_null.loc[k, "W0"], salm_null.loc[k,"W1"], salm_null.loc[k,"W2"]]
        usaf0 = salm_null.loc[k, "USAF0"]
        wban0 = salm_null.loc[k, "WBAN0"]
        filename0 = str(usaf0) + str(wban0) + "-" + str(yr) + "DAILY.csv"
        df0 = pd.read_csv(path + filename0)

        usaf1 = salm_null.loc[k, "USAF1"]
        wban1 = salm_null.loc[k, "WBAN1"]
        filename1 = str(usaf1) + str(wban1) + "-" + str(yr) + "DAILY.csv"
        df1 = pd.read_csv(path + filename1)

        usaf2 = salm_null.loc[k, "USAF2"]
        wban2 = salm_null.loc[k, "WBAN2"]
        filename2 = str(usaf2) + str(wban2) + "-" + str(yr) + "DAILY.csv"
        df2 = pd.read_csv(path + filename2)
        
        TEMP3 = [] ; MAX3 = [] ; MIN3 = [] ; DEWP3 = [] ; SLP3 = [] ; STP3 = [] ; WDSP3 = [] ; MXSPD3 = [];
        PRCP3 = [] ; SNDP3 = [];
        Fog3 = [] ; Rain3 = [] ; Snow3 = [] ; Hail3 = [] ; Thunder3 = [] ; Tornado3 = [];
        
        if(df0.DATE.str.contains(str(d)[:10]).any()):
            e = df0[df0.DATE == str(d)[:10]].index.tolist()[0]
            if(df0.loc[e, "TEMP"] < 9999):
                TEMP3.append(df0.loc[e, "TEMP"]) 
            if(df0.loc[e, "MAX"] < 9999):
                MAX3.append(df0.loc[e, "MAX"])
            if(df0.loc[e, "MIN"] < 9999):
                MIN3.append(df0.loc[e, "MIN"])
            if(df0.loc[e, "DEWP"] < 9999):
                DEWP3.append(df0.loc[e, "DEWP"])
            if(df0.loc[e, "SLP"] < 9999):
                SLP3.append(df0.loc[e, "SLP"])
            if(df0.loc[e, "STP"] < 9999):
                STP3.append(df0.loc[e, "STP"])
            if(df0.loc[e, "WDSP"] < 999):
                WDSP3.append(df0.loc[e, "WDSP"])
            if(df0.loc[e, "MXSPD"] < 998):
                MXSPD3.append(df0.loc[e, "MXSPD"])
            if(df0.loc[e, "PRCP"] < 99):
                PRCP3.append(df0.loc[e, "PRCP"])
            if(df0.loc[e, "SNDP"] == 999.9):
                SNDP3.append(0)
            else:
                SNDP3.append(df0.loc[e, "SNDP"])
            
            if((df0.loc[e, "FRSHTT"]%10) > 0):# 个位
                Tornado3.append(int(1))
            else:
                Tornado3.append(int(0))
            if(df0.loc[e, "FRSHTT"]//10%10 > 0):#十位
                Thunder3.append(int(1))       
            else:
                Thunder3.append(int(0))       
            if((df0.loc[e, "FRSHTT"]//100%10) > 0):#百位
                Hail3.append(int(1))
            else:
                Hail3.append(int(0))
            if((df0.loc[e, "FRSHTT"]//1000%10) > 0):#千位
                Snow3.append(int(1))
            else:
                Snow3.append(int(0))
            if((df0.loc[e, "FRSHTT"]//10000 % 10) > 0):
                Rain3.append(int(1))        
            else:
                Rain3.append(int(0))
            if((df0.loc[e, "FRSHTT"]//100000 % 10) > 0):
                Fog3.append(int(1))
            else:
                Fog3.append(int(0))
        else:
            w[0] = 0
        
 #df1       
        if(df1.DATE.str.contains(str(d)[:10]).any()):
            a = df1[df1.DATE == str(d)[:10]].index.tolist()[0]
            if(df1.loc[a, "TEMP"] < 9999):
                TEMP3.append(df1.loc[a, "TEMP"]) 
            if(df1.loc[a, "MAX"] < 9999):
                MAX3.append(df1.loc[a, "MAX"])
            if(df1.loc[a, "MIN"] < 9999):
                MIN3.append(df1.loc[a, "MIN"])
            if(df1.loc[a, "DEWP"] < 9999):
                DEWP3.append(df1.loc[a, "DEWP"])
            if(df1.loc[a, "SLP"] < 9999):
                SLP3.append(df1.loc[a, "SLP"])
            if(df1.loc[a, "STP"] < 9999):
                STP3.append(df1.loc[a, "STP"])
            if(df1.loc[a, "WDSP"] < 999):
                WDSP3.append(df1.loc[a, "WDSP"])
            if(df1.loc[a, "MXSPD"] < 998):
                MXSPD3.append(df1.loc[a, "MXSPD"])
            if(df1.loc[a, "PRCP"] < 99):
                PRCP3.append(df1.loc[a, "PRCP"])
            if(df1.loc[a, "SNDP"] == 999.9):
                SNDP3.append(0)
            else:
                SNDP3.append(df1.loc[a, "SNDP"])
            if((df1.loc[a, "FRSHTT"]%10) > 0):# 个位
                Tornado3.append(int(1))
            else:
                Tornado3.append(int(0))
            if(df1.loc[a, "FRSHTT"]//10%10 > 0):#十位
                Thunder3.append(int(1))       
            else:
                Thunder3.append(int(0))       
            if((df1.loc[a, "FRSHTT"]//100%10) > 0):#百位
                Hail3.append(int(1))
            else:
                Hail3.append(int(0))
            if((df1.loc[a, "FRSHTT"]//1000%10) > 0):#千位
                Snow3.append(int(1))
            else:
                Snow3.append(int(0))
            if((df1.loc[a, "FRSHTT"]//10000 % 10) > 0):
                Rain3.append(int(1))        
            else:
                Rain3.append(int(0))
            if((df1.loc[a, "FRSHTT"]//100000 % 10) > 0):
                Fog3.append(int(1))
            else:
                Fog3.append(int(0))
        else:
            w[1] = 0
#df2               
        if(df2.DATE.str.contains(str(d)[:10]).any()):
            b = df2[df2.DATE == str(d)[:10]].index.tolist()[0]
            if(df2.loc[b, "TEMP"] < 9999):
                TEMP3.append(df2.loc[b, "TEMP"]) 
            if(df2.loc[b, "MAX"] < 9999):
                MAX3.append(df2.loc[b, "MAX"])
            if(df2.loc[b, "MIN"] < 9999):
                MIN3.append(df2.loc[b, "MIN"])
            if(df2.loc[b, "DEWP"] < 9999):
                DEWP3.append(df2.loc[b, "DEWP"])
            if(df2.loc[b, "SLP"] < 9999):
                SLP3.append(df2.loc[b, "SLP"])
            if(df2.loc[b, "STP"] < 9999):
                STP3.append(df2.loc[b, "STP"])
            if(df2.loc[b, "WDSP"] < 999):
                WDSP3.append(df2.loc[b, "WDSP"])
            if(df2.loc[b, "MXSPD"] < 998):
                MXSPD3.append(df2.loc[b, "MXSPD"])
            if(df2.loc[b, "PRCP"] < 99):
                PRCP3.append(df2.loc[b, "PRCP"])
            if(df2.loc[b, "SNDP"] == 999.9):
                SNDP3.append(0)
            else:
                SNDP3.append(df2.loc[b, "SNDP"])
            if((df2.loc[b, "FRSHTT"]%10) > 0):# 个位
                Tornado3.append(int(1))
            else:
                Tornado3.append(int(0))
            if(df2.loc[b, "FRSHTT"]//10%10 > 0):#十位
                Thunder3.append(int(1))       
            else:
                Thunder3.append(int(0))       
            if((df2.loc[b, "FRSHTT"]//100%10) > 0):#百位
                Hail3.append(int(1))
            else:
                Hail3.append(int(0))
            if((df2.loc[b, "FRSHTT"]//1000%10) > 0):#千位
                Snow3.append(int(1))
            else:
                Snow3.append(int(0))
            if((df2.loc[b, "FRSHTT"]//10000 % 10) > 0):
                Rain3.append(int(1))        
            else:
                Rain3.append(int(0))
            if((df2.loc[b, "FRSHTT"]//100000 % 10) > 0):
                Fog3.append(int(1))
            else:
                Fog3.append(int(0))
        else:
            w[2] = 0
        
        for j in range(3):
            w[j] = w[j]/(w[0] + w[1] + w[2])
        w.remove(0)
        
        if not(len(TEMP3) == 0):
            TEMP2.append(sum(np.multiply(w, TEMP3)))
        if not(len(MAX3) == 0):
            MAX2.append(sum(np.multiply(w, MAX3)))
        if not (len(MIN3) == 0):
            MIN2.append(sum(np.multiply(w, MIN3)))
        if not (len(DEWP3) == 0):
            DEWP2.append(sum(np.multiply(w, DEWP3)))
        if not (len(SLP3) == 0):
            SLP2.append(sum(np.multiply(w, SLP3)))
        if not (len(STP3) == 0):
            STP2.append(sum(np.multiply(w,STP3)))
        if not (len(WDSP3) == 0):
            WDSP2.append(sum(np.multiply(w, WDSP3)))
        if not (len(MXSPD3) == 0):
            MXSPD2.append(sum(np.multiply(w, MXSPD3)))
        if not (len(PRCP3) == 0):
            PRCP2.append(sum(np.multiply(w,PRCP3)))
        if not (len(SNDP3) == 0):
            SNDP2.append(sum(np.multiply(w, SNDP3)))
        if not (len(Fog3) == 0):
            Fog2.append(sum(np.multiply(w, Fog3)))
        if not (len(Rain3) == 0):
            Rain2.append(sum(np.multiply(w, Rain3)))
        if not (len(Snow3) == 0):
            Snow2.append(sum(np.multiply(w, Snow3)))
        if not (len(Hail3) == 0):
            Hail2.append(sum(np.multiply(w, Hail3)))
        if not (len(Thunder3) == 0):
            Thunder2.append(sum(np.multiply(w, Thunder3)))
        if not (len(Tornado3) == 0):
            Tornado2.append(sum(np.multiply(w, Tornado3)))
        d = d + delta
    
    salm.loc[index, "TEMP"] = np.mean(TEMP2); 
    salm.loc[index, "MAX"] = np.mean(MAX2); 
    salm.loc[index, "MIN"] = np.mean(MIN2);
    salm.loc[index, "DEWP"] = np.mean(DEWP2); 
    salm.loc[index, "SLP"] = np.mean(SLP2); 
    salm.loc[index, "STP"] = np.mean(STP2);
    salm.loc[index, "WDSP"] = np.mean(WDSP2); 
    salm.loc[index, "MXSPD"] = np.mean(MXSPD2); 
    salm.loc[index, "PRCP"] = np.mean(PRCP2);
    salm.loc[index, "SNDP"] = np.mean(SNDP2); 
    salm.loc[index, "Fog"] = np.mean(Fog2); 
    salm.loc[index, "Rain or Drizzle"] = np.mean(Rain2);
    salm.loc[index, "Snow or Ice Pellets"] = np.mean(Snow2); 
    salm.loc[index, "Hail"] = np.mean(Hail2); 
    salm.loc[index, "Thunder"] = np.mean(Thunder2); 
    salm.loc[index, "Tornado or Funnel Cloud"] = np.mean(Tornado2); 
    
    
    

In [322]:
salm.isnull().sum()

Unnamed: 0                   0
CDCID                        0
GenusName                    0
InitialExposure              0
LastExposure                 0
TotalCases                   0
Exposurestate                1
ExposureState                0
ExposureCounty1              0
ExposureCounty2            841
ExposureCounty3            867
ExposureCounty4            890
ExposureCounty5            901
ExposureCounty6            909
ExposureCounty7            919
ExposureCounty8            924
ExposureCounty9            927
ExposureCounty10           928
ExposureCounty11           931
ExposureCounty12           932
ExposureCounty13           936
ExposureCounty14           938
ExposureCounty15           938
ExposureCounty16           938
ExposureCounty17           939
ExposureCounty18           939
ExposureCounty19           940
TEMP                         0
MAX                          0
MIN                          0
DEWP                        11
SLP                        128
STP     

In [324]:
salm.to_csv('/Users/evelyn/Desktop/S1.csv')

In [318]:
salm.to_csv('/Users/evelyn/Desktop/Salmonella.csv')